In [76]:
batch_size=25
num_batches=4
num_records=1000
## Each task handles 25*4 records, there should be 10 partitions overall to process 1000 records.

def getBatchSentiment(input_list):
    j=[i[1] for i in input_list]
    client = boto3.client('comprehend',region_name='us-west-2')
    for i in range(num_batches-1):
       k=j[batch_size*i:batch_size*(i+1)]
       print (k)
       response = client.batch_detect_sentiment(
          TextList=k,
          LanguageCode='en')
       #print response['ResultList']
       for r in response['ResultList']:
            index=batch_size*i+r['Index']
            print (str(index) + ":" +r['Sentiment'])
            input_list[index]=(input_list[index][0],input_list[index][1],r['Sentiment'])
    return input_list

In [77]:
import random, boto3
reviews=['I like this product a lot','Too expensive. A waste of money']

items=[]
for i in range(num_records):
    items.append((i,random.choice(reviews),''))

In [78]:
rdd=sc.parallelize(items)
df=rdd.toDF(['review_id','review_text','sentiment'])

In [79]:
df.show()
df.cache()

+---------+--------------------+---------+
|review_id|         review_text|sentiment|
+---------+--------------------+---------+
|        0|Too expensive. A ...|         |
|        1|Too expensive. A ...|         |
|        2|I like this produ...|         |
|        3|Too expensive. A ...|         |
|        4|Too expensive. A ...|         |
|        5|I like this produ...|         |
|        6|Too expensive. A ...|         |
|        7|Too expensive. A ...|         |
|        8|I like this produ...|         |
|        9|Too expensive. A ...|         |
|       10|I like this produ...|         |
|       11|Too expensive. A ...|         |
|       12|Too expensive. A ...|         |
|       13|I like this produ...|         |
|       14|Too expensive. A ...|         |
|       15|I like this produ...|         |
|       16|Too expensive. A ...|         |
|       17|Too expensive. A ...|         |
|       18|Too expensive. A ...|         |
|       19|Too expensive. A ...|         |
+---------+

In [80]:
record_count=df.count()
print record_count
df1=df.repartition(record_count/(num_batches*batch_size))
df1.rdd.getNumPartitions()

1000
10

In [81]:
#Sort each partition by review id and convert to RDD
rdd1=df1.sortWithinPartitions(['review_id'],ascending=True).rdd
#Map each Tuple Row to fields and convert to list.
rdd2=rdd1.map(lambda line: tuple([str(x) for x in line])).glom()
#invole batch sentiment and map back to records.
output=rdd2.map(lambda l: getBatchSentiment(l)).flatMap(lambda x:x).collect()
print (output)

[('9', 'Too expensive. A waste of money', u'NEGATIVE'), ('19', 'Too expensive. A waste of money', u'NEGATIVE'), ('29', 'Too expensive. A waste of money', u'NEGATIVE'), ('39', 'Too expensive. A waste of money', u'NEGATIVE'), ('49', 'I like this product a lot', u'POSITIVE'), ('59', 'Too expensive. A waste of money', u'NEGATIVE'), ('69', 'Too expensive. A waste of money', u'NEGATIVE'), ('79', 'I like this product a lot', u'POSITIVE'), ('89', 'Too expensive. A waste of money', u'NEGATIVE'), ('99', 'I like this product a lot', u'POSITIVE'), ('109', 'I like this product a lot', u'POSITIVE'), ('119', 'I like this product a lot', u'POSITIVE'), ('129', 'Too expensive. A waste of money', u'NEGATIVE'), ('139', 'I like this product a lot', u'POSITIVE'), ('149', 'Too expensive. A waste of money', u'NEGATIVE'), ('159', 'Too expensive. A waste of money', u'NEGATIVE'), ('169', 'Too expensive. A waste of money', u'NEGATIVE'), ('179', 'Too expensive. A waste of money', u'NEGATIVE'), ('189', 'Too expensi

In [82]:
len(output)

1000